In [ ]:
from platform import python_version
print(python_version())

we will show you how to access market data using Polygon's Stock API and the Python programming language. Polygon is a financial data platform that provides both real-time and historical market data for Stocks, Options, Forex, and Crypto. With access to this information, developers, investors, and financial institutions can gain valuable insights and make informed decisions.
#https://polygon.io/blog/polygon-io-with-python-for-stock-market-data/

#“polygon-api-client” client library for accessing market data
#"requests" for REST API requests, and "websockets" for real-time data streaming.
#"websockets" for real-time data streaming - This feature comes under paid subscription

In [ ]:
#!pip install polygon-api-client
from polygon import RESTClient # We need Python version 3.8+ to install polygon package
from datetime import datetime
import pandas as pd
import pprint


In [ ]:
# API Key 
polygon_api_key="ACCESS-Key"
client = RESTClient(polygon_api_key)

In [ ]:
#To get a list exchanges in USA
exchanges = pd.DataFrame(client.get_exchanges(asset_class='stocks',
                                              locale='us'))

In [ ]:
exchanges.head()

In [ ]:
tickers = []
for t in client.list_tickers(market="stocks", type="CS", active=True, limit=1000):
    tickers.append(t)
print(tickers)

In [ ]:
df_tickers =pd.DataFrame(tickers)

In [ ]:
df_tickers.head()

In [ ]:
from datetime import datetime,timedelta
today_date = datetime.now().strftime("%Y-%m-%d")
today_date_minus_1=datetime.now() + timedelta(days=-1) 
yesterday=today_date_minus_1.strftime("%Y-%m-%d")

today_date_minus_30=datetime.now() + timedelta(days=-30) 

start_date=today_date_minus_30.strftime("%Y-%m-%d")
end_date= datetime.now() + timedelta(days=-1) 
end_date=end_date.strftime("%Y-%m-%d")

In [ ]:
# Get the daily open, high, low, and close (OHLC) for the entire market.



mktData = client.get_grouped_daily_aggs(date=end_date)

# pprint (short for "pretty-print") is a module that provides a more human-
# readable output format for data structures.
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(mktData)

In [ ]:
# getting  details for a ticker
details = client.get_ticker_details("AAPL")
print(details)
#df_details_ticker=pd.DataFrame(details)

In [ ]:
# Get aggregate bars for a ticker over a given date range in custom time window sizes.
# We will use this data to send it to Kafka Producer

ticker='AAPL'
aggs = client.get_aggs(
    ticker=ticker,
    multiplier=1, 
    timespan="day",
    from_=start_date,
    to=end_date,
)

df_apple_30days_data=pd.DataFrame(aggs)

In [ ]:
df_apple_30days_data['timestamp'] = df_apple_30days_data['timestamp'].apply(
                          lambda x: pd.to_datetime(x*1000000))

In [ ]:
df_apple_30days_data.head()

In [ ]:
# will json data to Producer
df_apple_30days_data.sample(1).to_dict(orient="records")[0]

# Send Data to Producer

In [ ]:
# !pip install kafka-python
import pandas as pd
from kafka import KafkaProducer 
from time import sleep
from json import dumps
import json

In [ ]:
producer = KafkaProducer(bootstrap_servers=['<public-ip>:9092'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [ ]:
topic_name='demo_test'

while True:
    dict_stock = df_apple_30days_data.sample(1).to_dict(orient="records")[0]
    producer.send(topic_name, value=dict_stock)
    sleep(1)

In [ ]:
producer.flush() #clear data from kafka server